
<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/derar-alhussein/Databricks-Certified-Data-Engineer-Professional/main/Includes/images/books.png" width="60%">
</div>

In [0]:
%run ../Includes/Copy-Datasets

Data catalog: workspace
Schema: bookstore_eng_pro


In [0]:
%sql
-- NOTE: This cell contains the SQL query for your reference, and won't work if run directly.
-- The query is used below in the type2_upsert() function as part of the foreachBatch call.

MERGE INTO books_silver
USING (
    SELECT updates.book_id as merge_key, updates.*
    FROM updates

    UNION ALL

    SELECT NULL as merge_key, updates.*
    FROM updates
    JOIN books_silver ON updates.book_id = books_silver.book_id
    WHERE books_silver.current = true AND updates.price <> books_silver.price
  ) staged_updates
ON books_silver.book_id = merge_key 
WHEN MATCHED AND books_silver.current = true AND books_silver.price <> staged_updates.price THEN
  UPDATE SET current = false, end_date = staged_updates.updated
WHEN NOT MATCHED THEN
  INSERT (book_id, title, author, price, current, effective_date, end_date)
  VALUES (staged_updates.book_id, staged_updates.title, staged_updates.author, staged_updates.price, true, staged_updates.updated, NULL)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8171290041955501>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', "-- NOTE: This cell contains the SQL query for your reference, and won't work if run directly.\n-- The query is used below in the type2_upsert() function as part of the foreachBatch call.\n\nMERGE INTO books_silver\nUSING (\n    SELECT updates.book_id as merge_key, updates.*\n    FROM updates\n\n    UNION ALL\n\n    SELECT NULL as merge_key, updates.*\n    FROM updates\n    JOIN books_silver ON updates.book_id = books_silver.book_id\n    WHERE books_silver.current = true AND updates.price <> books_silver.price\n  ) staged_updates\nON books_silver.book_id = merge_key \nWHEN MATCHED AND books_silver.current = true AND books_silver.price <> staged_updates.price THEN\n  UPDATE SET current = false, end_date = staged_updates.updated\nWHEN NOT MATCHED THEN\

In [0]:
def type2_upsert(microBatchDF, batch):
    microBatchDF.createOrReplaceTempView("updates")
    
    sql_query = """
        MERGE INTO books_silver
        USING (
            SELECT updates.book_id as merge_key, updates.*
            FROM updates

            UNION ALL

            SELECT NULL as merge_key, updates.*
            FROM updates
            JOIN books_silver ON updates.book_id = books_silver.book_id
            WHERE books_silver.current = true AND updates.price <> books_silver.price
          ) staged_updates
        ON books_silver.book_id = merge_key 
        WHEN MATCHED AND books_silver.current = true AND books_silver.price <> staged_updates.price THEN
          UPDATE SET current = false, end_date = staged_updates.updated
        WHEN NOT MATCHED THEN
          INSERT (book_id, title, author, price, current, effective_date, end_date)
          VALUES (staged_updates.book_id, staged_updates.title, staged_updates.author, staged_updates.price, true, staged_updates.updated, NULL)
    """
    
    microBatchDF.sparkSession.sql(sql_query)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS books_silver
(book_id STRING, title STRING, author STRING, price DOUBLE, current BOOLEAN, effective_date TIMESTAMP, end_date TIMESTAMP)

In [0]:
def process_books():
    schema = "book_id STRING, title STRING, author STRING, price DOUBLE, updated TIMESTAMP"
 
    query = (spark.readStream
                    .table("bronze")
                    .filter("topic = 'books'")
                    .select(F.from_json(F.col("value").cast("string"), schema).alias("v"))
                    .select("v.*")
                 .writeStream
                    .foreachBatch(type2_upsert)
                    .option("checkpointLocation", f"{bookstore.checkpoint_path}/books_silver")
                    .trigger(availableNow=True)
                    .start()
            )
    
    query.awaitTermination()
    
process_books()

In [0]:
books_df = spark.read.table("books_silver").orderBy("book_id", "effective_date")
display(books_df)

book_id,title,author,price,current,effective_date,end_date
B01,The Soul of a New Machine,Tracy Kidder,49.0,true,2021-11-07T17:11:33.507Z,null
B02,Learning JavaScript Design Patterns,Addy Osmani,28.0,true,2021-11-07T17:11:33.507Z,null
B03,Make Your Own Neural Network,Tariq Rashid,35.0,true,2021-11-07T17:11:33.507Z,null
B04,Robot Dynamics and Control,Mark W. Spong,20.0,true,2021-11-08T17:12:05.419Z,null
B05,Fluent Python,Luciano Ramalho,47.0,true,2021-11-08T17:12:05.419Z,null
B06,Deep Learning with Python,François Chollet,22.0,true,2021-11-08T17:12:05.419Z,null
B07,The Hundred-Page Machine Learning,Andriy Burkov,33.0,true,2021-11-09T17:11:49.506Z,null
B08,Quantum Computing for Everyone,Chris Bernhardt,41.0,true,2021-11-09T17:11:49.506Z,null
B09,Advanced Data Structures,Peter Brass,24.0,true,2021-11-09T17:11:49.506Z,null
B10,Beginning Database Design Solutions,Rod Stephens,44.0,true,2021-11-10T16:36:31.241Z,null


In [0]:
bookstore.load_books_updates()
bookstore.process_bronze()
process_books()

Loading books-updates-streaming-01.json file to the bookstore dataset


In [0]:
books_df = spark.read.table("books_silver").orderBy("book_id", "effective_date")
display(books_df)

book_id,title,author,price,current,effective_date,end_date
B01,The Soul of a New Machine,Tracy Kidder,49.0,true,2021-11-07T17:11:33.507Z,null
B02,Learning JavaScript Design Patterns,Addy Osmani,28.0,true,2021-11-07T17:11:33.507Z,null
B03,Make Your Own Neural Network,Tariq Rashid,35.0,true,2021-11-07T17:11:33.507Z,null
B04,Robot Dynamics and Control,Mark W. Spong,20.0,true,2021-11-08T17:12:05.419Z,null
B05,Fluent Python,Luciano Ramalho,47.0,true,2021-11-08T17:12:05.419Z,null
B06,Deep Learning with Python,François Chollet,22.0,true,2021-11-08T17:12:05.419Z,null
B07,The Hundred-Page Machine Learning,Andriy Burkov,33.0,true,2021-11-09T17:11:49.506Z,null
B08,Quantum Computing for Everyone,Chris Bernhardt,41.0,true,2021-11-09T17:11:49.506Z,null
B09,Advanced Data Structures,Peter Brass,24.0,true,2021-11-09T17:11:49.506Z,null
B10,Beginning Database Design Solutions,Rod Stephens,44.0,true,2021-11-10T16:36:31.241Z,null



<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/derar-alhussein/Databricks-Certified-Data-Engineer-Professional/main/Includes/images/current_books.png" width="60%">
</div>

In [0]:
%sql
CREATE OR REPLACE TABLE current_books
AS SELECT book_id, title, author, price
   FROM books_silver
   WHERE current IS TRUE

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT *
FROM current_books
ORDER BY book_id

book_id,title,author,price
B01,The Soul of a New Machine,Tracy Kidder,49.0
B02,Learning JavaScript Design Patterns,Addy Osmani,28.0
B03,Make Your Own Neural Network,Tariq Rashid,35.0
B04,Robot Dynamics and Control,Mark W. Spong,20.0
B05,Fluent Python,Luciano Ramalho,47.0
B06,Deep Learning with Python,François Chollet,22.0
B07,The Hundred-Page Machine Learning,Andriy Burkov,33.0
B08,Quantum Computing for Everyone,Chris Bernhardt,41.0
B09,Advanced Data Structures,Peter Brass,24.0
B10,Beginning Database Design Solutions,Rod Stephens,44.0
